# ESECUZIONE

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 28 17:45:00 2020

Modulo per calcolare le varie metriche/attributi necessari
per stabilire se un testo è stato scritto da un autore

@author: michele
"""
'''import pyspark
sc = pyspark.SparkContext('local[*]')'''

def prob_of_comma(RDD_sentences_data, text_length):
    return (RDD_sentences_data.flatMap(lambda x: x)
           .filter(lambda x: "," in x)
           .count()
           ) / text_length

def prob_distr_of_sentence_length(RDD_sen_len):
    tot = len(getCollection(RDD_sen_len))
    
    return (RDD_sen_len.map(lambda x: (x,1))
            .reduceByKey(lambda a,b: a+b)
            .map(lambda x: (x[0], x[1]/tot))
            .sortBy(lambda x: -x[1])
           )

def prob_of_the_most_freq_sentence_len(RDD_sen_len):
    return prob_distr_of_sentence_length(RDD_sen_len).take(1)

def sentence_lengths(RDD):
    # OPERAZIONI PRELIMINARI SUL TESTO
    text = RDD.flatMap(lambda x: x).reduce(lambda a,b: a + ' ' + b) # METTO TUTTO IL TESTO IN UNA STRINGA UNICA
    text = text.replace("?", ".") # ? TERMINA UNA FRASE
    text = text.replace("!", ".") # ! TERMINA UNA FRASE
    text = text.split('. ') # SPLITTO QUANDO TROVO UN CARATTERE CHE TERMINA UNA FRASE (. SEGUITO DA UNO SPAZIO)
    
    return (sc.parallelize(text)
            .map(lambda x: len(x.split(' ')))
            .collect()
           ) # PER OGNI FRASE TROVATA CONTO LE SUE PAROLE

def prob_of_the_most_common_word_x(RDD_word_counter, text_len):
    prep_art = open("preposizioni_e_articoli.txt").read().splitlines()
    
    return sc.parallelize(prob_distr_of_30_most_common_words(RDD_word_counter, text_len)
                          .filter(lambda x: x[0] not in prep_art)
                          .take(1)
                         )

def prob_distr_of_30_most_common_words_x(RDD_word_counter, text_len):
    
    prep_art = open("preposizioni_e_articoli.txt").read().splitlines()
    
    # probability distribution
    return sc.parallelize(RDD_word_counter.filter(lambda x: x[0] not in prep_art)
                          .map(lambda x: (x[0], x[1]/text_len))
                          .take(30)
                         )

def avg_distance_consec_appear(RDD, MCW):
    vect_dis = distance_consec_appear(RDD, MCW)
    
    _sum = 0
    for i in vect_dis:
        _sum += i
    
    return _sum/len(vect_dis)

def max_distance_consec_appear(RDD, MCW):
    return max(distance_consec_appear(RDD, MCW))

def min_distance_consec_appear(RDD, MCW):
    return min(distance_consec_appear(RDD, MCW))

def distance_consec_appear(RDD, MCW):
    if MCW == ",":
        vect_pos = (RDD.flatMap(lambda x:x)
                    .zipWithIndex()
                    .filter(lambda x: "," in x[0])
                    .map(lambda x: [x[1]])
                    .reduce(lambda a,b: a+b)
                   )
    else:
        vect_pos = (RDD.flatMap(lambda x:x)
                    .zipWithIndex()
                    .filter(lambda x:x[0] == MCW)
                    .map(lambda x: [x[1]])
                    .reduce(lambda a,b: a+b)
                   )
    
    vect_dis = []
    
    for i in range(1, len(vect_pos)):
        vect_dis.append(vect_pos[i] - vect_pos[i-1])
    
    return vect_dis


def prob_of_the_most_common_word(RDD_word_counter, text_len):
    return sc.parallelize(prob_distr_of_30_most_common_words(RDD_word_counter, text_len)
                          .filter(lambda x: x[0] != "and" and x[0] != "the")
                          .take(1)
                         )

def prob_of_The(RDD_word_counter, text_len):
    return (RDD_word_counter.filter(lambda x: x[0] == "the")
           .map(lambda x: (x[0], x[1]/text_len))
           )

def prob_distr_of_30_most_common_words(RDD_word_counter, text_len):
    # probability distribution
    return sc.parallelize(RDD_word_counter.map(lambda x: (x[0], x[1]/text_len)).take(30))

def hentropy(RDD_word_counter, text_len):
    import math
    
    return (RDD_word_counter.map(lambda x: ("hentropy", (x[1]/text_len) * math.log2(x[1]/text_len)))
                            .reduceByKey(lambda a,b: a+b)
                            .map(lambda x: -x[1])    # entropia ha segno negativo
           )

def text_length_in_words(RDD_word_counter):
    
    # word_counter: [("word1", 100), ...]
    
    return (RDD_word_counter.map(lambda x: x[1])
                          .reduce(lambda a,b: a+b)
           )

def word_counter(RDD):
    word_counter = (RDD.flatMap(lambda x: x)
                .map(lambda x: (x,1))
                .reduceByKey(lambda a,b: a+b)
                .sortBy(lambda x: -x[1])
               )
    return word_counter, len(word_counter.collect())


def getCollection(RDD):
    return RDD.collect()

def getValue(RDD):
    return RDD.collect()[0]


def remove_number_some_punctuation_marks(row):

    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if not ('0' <= char <= '9' or char == '"'):
            res += char

    return res

def remove_number_punctuation_marks(row):
    
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    
    res = ""
    
    for char in lowercase:
        if 'a' <= char <= 'z' or char == ' ' or char == '-' or char == "'":
            res += char

    return res

def load_file_without_punctuations_marks(filepath):
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def load_file_without_number(filepath):
    # caricamento del dataset
    raw_text = sc.textFile(filepath)

    # rimuoviamo i numeri e i segni di punteggiatura
    
    return (raw_text.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_some_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))        # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

def generate_metrics(file_in):
    res = {}
    
    print("Caricamento del file ... ", end=" ")
    data = load_file_without_punctuations_marks(file_in)
    print("caricamento completato")
    
    # POSIAMO I DATI NELLA CACHE
    data.persist()
    
    print("Calcolo le metriche attendere ... ", end=" ")
    
    RDD_word_counter, vocabulary_size = word_counter(data)
    text_length = text_length_in_words(RDD_word_counter)
    RDD_hentropy = hentropy(RDD_word_counter, text_length)

    RDD_prob_distr_of_30 = prob_distr_of_30_most_common_words(RDD_word_counter, text_length)
    RDD_prob_the_most_common_word = prob_of_the_most_common_word(RDD_word_counter, text_length)
    RDD_prob_the = prob_of_The(RDD_word_counter, text_length)


    avg_dist_consec_MCW = avg_distance_consec_appear(data, getValue(RDD_prob_the_most_common_word)[0])
    min_dist_consec_MCW = min_distance_consec_appear(data, getValue(RDD_prob_the_most_common_word)[0])
    max_dist_consec_MCW = max_distance_consec_appear(data, getValue(RDD_prob_the_most_common_word)[0])

    avg_dist_consec_the = avg_distance_consec_appear(data, "the")
    min_dist_consec_the = min_distance_consec_appear(data, "the")
    max_dist_consec_the = max_distance_consec_appear(data, "the")
    
    # genero MCWx da passare come parametro alle funzioni che calcolano le distanze
    MCWx = getValue(prob_of_the_most_common_word_x(RDD_word_counter, text_length))[0]
    
    avg_dist_consec_MCWx = avg_distance_consec_appear(data, MCWx)
    min_dist_consec_MCWx = min_distance_consec_appear(data, MCWx)
    max_dist_consec_MCWx = max_distance_consec_appear(data, MCWx)

    # CONSIDERIAMO IL TESTO CON I SEGNI DI PUNTEGGIATURA
    
    # genero la lista delle frasi del testo
    sentences_data = load_file_without_number(file_in)
    sentences_data.persist()
    
    #min(lengths), max(lengths), sum(lengths)/len(lengths)
    sen_lengths = sentence_lengths(sentences_data)
    
    # trasporto in RDD
    RDD_sen_lengths = sc.parallelize(sen_lengths)
    RDD_sen_lengths.persist()
    
    # probabilità della lunghezza della frase più frequente
    prob_most_freq_sen = prob_of_the_most_freq_sentence_len(RDD_sen_lengths)
    
    # distribuzione di probabilità sulla lunghezza delle frase 
    prob_distr_freq_sen = prob_distr_of_sentence_length(RDD_sen_lengths)
    
    # probabilità di incontrare una virgola
    p_comma = prob_of_comma(sentences_data, text_length)

    avg_dist_consec_comma = avg_distance_consec_appear(sentences_data, ",")
    min_dist_consec_comma = min_distance_consec_appear(sentences_data, ",")
    max_dist_consec_comma = max_distance_consec_appear(sentences_data, ",")
    
    
    # creiamo la struttura dati
    res['vocabulary_size'] = vocabulary_size
    res['text_length'] = text_length
    res['V_T'] = vocabulary_size/text_length
    res['hentropy'] = getValue(RDD_hentropy)
    
    res['prob_distr_of_30'] = getCollection(RDD_prob_distr_of_30)
    res['prob_of_the_most_common_word'] = getValue(RDD_prob_the_most_common_word)[1]
    res['prob_of_the'] = getValue(RDD_prob_the)[1]
    res['prob_of_comma'] = p_comma
    
    res['avg_dist_consec_MCW'] = avg_dist_consec_MCW
    res['min_dist_consec_MCW'] = min_dist_consec_MCW
    res['max_dist_consec_MCW'] = max_dist_consec_MCW
    
    res['avg_dist_consec_the'] = avg_dist_consec_the
    res['min_dist_consec_the'] = min_dist_consec_the
    res['max_dist_consec_the'] = max_dist_consec_the
    
    res['avg_dist_consec_MCWx'] = avg_dist_consec_MCWx
    res['min_dist_consec_MCWx'] = min_dist_consec_MCWx
    res['max_dist_consec_MCWx'] = max_dist_consec_MCWx
    
    res['avg_dist_consec_comma'] = avg_dist_consec_comma
    res['min_dist_consec_comma'] = min_dist_consec_comma
    res['max_dist_consec_comma'] = max_dist_consec_comma
    
    res['avg_sentence_len'] = sum(sen_lengths)/len(sen_lengths)
    res['max_sentence_len'] = max(sen_lengths)
    res['min_sentence_len'] = min(sen_lengths)
    
    res['prob_most_freq_sen'] = prob_most_freq_sen[0][1]
    res['prob_distr_freq_sen'] = getCollection(prob_distr_freq_sen)
    
    print("calcolo completato")
    
    return res
    
def save_metrics(file_in, file_out):
    '''
    save_metrics salva un dictionary in un file 
    Parameters
    ----------
    file_in : str
        path del file da analizzare
    file_out: str
        path del file in cui salvare la entry

    Returns
    -------
    None.
    '''
    import pickle

    entry = generate_metrics(file_in)
    
    with open(file_out, 'ab') as fout:
        pickle.dump(entry, fout, pickle.HIGHEST_PROTOCOL)
    
    print("Salvataggio completato")

def load_metrics(file_in):
    '''
    load_metrics dal file le diverse righe (dict()) 
    Parameters
    ----------
    file_in : str
        path del file da caricare

    Returns
    -------
    None.

    '''
    import pickle
    res = []
    with open(file_in, "rb") as fin:
        while True:
            try:
                res.append(pickle.load(fin))
            except EOFError:
                break
    return res

In [2]:
if __name__ == "__main__":
    print("Caricamento del file ... ", end=" ")
    data = load_file_without_punctuations_marks("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
    print("caricamento completato")
    
    # POSIAMO I DATI NELLA CACHE
    data.persist()
    
    print("Calcoliamo l'RDD del word_counter ... ", end=" ")
    RDD_word_counter, vocabulary_size = word_counter(data)
    print("calcolo completato")
    
    print("text_length_in_word ... ", end=" ")
    text_length = text_length_in_words(RDD_word_counter)
    print(text_length)
    
    print("Rapporto V/T: ", vocabulary_size/text_length)
    
    print("Calcolo entropia ... ", end=" ")
    RDD_hentropy = hentropy(RDD_word_counter, text_length)
    print(getValue(RDD_hentropy))
    

Caricamento del file ...  caricamento completato
Calcoliamo l'RDD del word_counter ...  calcolo completato
text_length_in_word ...  7653
Rapporto V/T:  0.20358029530902913
Calcolo entropia ...  8.605861060321123


In [3]:
    print("Calcolo della distribuzione di probabilità delle 30 parole più comuni ...", end=" ")
    RDD_prob_distr_of_30 = prob_distr_of_30_most_common_words(RDD_word_counter, text_length)
    print(getCollection(RDD_prob_distr_of_30), end="\n\n")
    
    print("Calcolo della probabilità di the ...", end=" ")
    RDD_prob_the = prob_of_The(RDD_word_counter, text_length)
    print(getValue(RDD_prob_the))
    
    print("Calcolo della probabilità della parola più comune escluso the e and ...", end=" ")
    RDD_prob_the_most_common_word = prob_of_the_most_common_word(RDD_word_counter, text_length)
    print(getValue(RDD_prob_the_most_common_word))
    

Calcolo della distribuzione di probabilità delle 30 parole più comuni ... [('the', 0.04494969293087678), ('i', 0.036717627074349925), ('and', 0.033320266562132494), ('to', 0.023650855873513656), ('of', 0.021298837057363126), ('a', 0.019730824513262774), ('my', 0.01620279628903698), ('that', 0.016072128577028617), ('in', 0.015941460865020254), ('was', 0.01528812230497844), ('said', 0.012152097216777734), ('as', 0.011106755520710831), ('he', 0.011106755520710831), ('at', 0.009930746112635568), ('but', 0.009669410688618842), ('you', 0.009277407552593753), ('for', 0.009146739840585392), ('me', 0.00862406899255194), ('it', 0.00862406899255194), ('had', 0.007840062720501764), ('with', 0.00718672416045995), ('not', 0.006533385600418136), ("o'conor", 0.006272050176401411), ('all', 0.006141382464393048), ('his', 0.005880047040376323), ('were', 0.00574937932836796), ('on', 0.005618711616359597), ('so', 0.004965373056317784), ('there', 0.004704037632301058), ('we', 0.004704037632301058)]

Calcolo

In [4]:
    print("Calcolo della distanza media fra consecutive MCW ...", end=" ")
    avg_dist_consec_MCW = avg_distance_consec_appear(data, getValue(RDD_prob_the_most_common_word)[0])
    print((avg_dist_consec_MCW))
    
    print("Calcolo della distanza minima fra consecutive MCW ...", end=" ")
    min_dist_consec_MCW = min_distance_consec_appear(data, getValue(RDD_prob_the_most_common_word)[0])
    print((min_dist_consec_MCW))
    
    print("Calcolo della distanza massima fra consecutive MCW ...", end=" ")
    max_dist_consec_MCW = max_distance_consec_appear(data, getValue(RDD_prob_the_most_common_word)[0])
    print((max_dist_consec_MCW))
    
    print("Calcolo della distanza media fra consecutive the ...", end=" ")
    avg_dist_consec_the = avg_distance_consec_appear(data, "the")
    print((avg_dist_consec_the))
    
    print("Calcolo della distanza minima fra consecutive the ...", end=" ")
    min_dist_consec_the = min_distance_consec_appear(data, "the")
    print((min_dist_consec_the))
    
    print("Calcolo della distanza massima fra consecutive the ...", end=" ")
    max_dist_consec_the = max_distance_consec_appear(data, "the")
    print((max_dist_consec_the))

Calcolo della distanza media fra consecutive MCW ... 27.15357142857143
Calcolo della distanza minima fra consecutive MCW ... 2
Calcolo della distanza massima fra consecutive MCW ... 177
Calcolo della distanza media fra consecutive the ... 22.244897959183675
Calcolo della distanza minima fra consecutive the ... 2
Calcolo della distanza massima fra consecutive the ... 115


In [7]:
    MCWx = getValue(prob_of_the_most_common_word_x(RDD_word_counter, text_length))[0]
    
    print("MCWx =", MCWx)
    
    print("Calcolo della distanza media fra consecutive MCWx ...", end=" ")
    avg_dist_consec_MCWx = avg_distance_consec_appear(data, MCWx)
    print((avg_dist_consec_MCWx))
    
    print("Calcolo della distanza minima fra consecutive MCWx ...", end=" ")
    min_dist_consec_MCWx = min_distance_consec_appear(data, MCWx)
    print((min_dist_consec_MCWx))
    
    print("Calcolo della distanza massima fra consecutive MCWx ...", end=" ")
    max_dist_consec_MCWx = max_distance_consec_appear(data, MCWx)
    print((max_dist_consec_MCWx))

MCWx = i
Calcolo della distanza media fra consecutive MCWx ... 27.15357142857143
Calcolo della distanza minima fra consecutive MCWx ... 2
Calcolo della distanza massima fra consecutive MCWx ... 177


In [34]:
    sentences_data = load_file_without_number("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")
    sentences_data.persist()
    
    sen_lengths = sentence_lengths(sentences_data)
    print("MSL:\t", max(lengths))
    print("ASL:\t", sum(lengths)/len(lengths))
    print("mSL:\t", min(lengths))
    
    RDD_sen_lengths = sc.parallelize(sen_lengths)
    
    prob_most_freq_sen = prob_of_the_most_freq_sentence_len(RDD_sen_lengths)
    print("PMSL:\t", prob_most_freq_sen)
    
    prob_distr_freq_sen = prob_distr_of_sentence_length(RDD_sen_lengths)
    print("PdSL:\t", getCollection(prob_distr_freq_sen), end="\n\n")

MSL:	 84
ASL:	 14.862135922330097
mSL:	 1
PMSL:	 [(7, 0.06407766990291262)]
PdSL:	 [(7, 0.06407766990291262), (2, 0.05631067961165048), (9, 0.04660194174757282), (10, 0.04660194174757282), (17, 0.04271844660194175), (12, 0.040776699029126215), (8, 0.038834951456310676), (16, 0.038834951456310676), (1, 0.038834951456310676), (15, 0.038834951456310676), (3, 0.038834951456310676), (6, 0.036893203883495145), (5, 0.03495145631067961), (11, 0.03495145631067961), (13, 0.03300970873786408), (4, 0.031067961165048542), (20, 0.02912621359223301), (25, 0.02330097087378641), (22, 0.02330097087378641), (18, 0.021359223300970873), (14, 0.021359223300970873), (29, 0.019417475728155338), (23, 0.019417475728155338), (19, 0.019417475728155338), (21, 0.017475728155339806), (24, 0.015533980582524271), (28, 0.013592233009708738), (26, 0.013592233009708738), (35, 0.011650485436893204), (30, 0.009708737864077669), (31, 0.009708737864077669), (27, 0.007766990291262136), (41, 0.005825242718446602), (37, 0.00582

In [50]:
    p_comma = prob_of_comma(sentences_data, text_length)
    print("Pcomma:\t", p_comma)
    
    print("max_dist_consec_appear_comma:\t", max_distance_consec_appear(sentences_data, ","))
    print("min_dist_consec_appear_comma:\t", min_distance_consec_appear(sentences_data, ","))
    print("avg_dist_consec_appear_comma:\t", avg_distance_consec_appear(sentences_data, ","))

NameError: name 'sentences_data' is not defined

In [51]:
    save_metrics("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt", "Anthony Trollope")
    save_metrics("datasets/Anthony Trollope___George Walker At Suez.txt", "Anthony Trollope")

Caricamento del file ...  caricamento completato
Calcolo le metriche attendere ...  calcolo completato
Salvataggio completato
Caricamento del file ...  caricamento completato
Calcolo le metriche attendere ...  calcolo completato
Salvataggio completato


In [10]:
    import statistics
    
    def mean_std_couple(_list, tot_el):
        for i in range(0, tot_el - len(_list)):
            _list.append(0)

        return (statistics.mean(_list), statistics.stdev(_list))
    
    
    analisi = load_metrics("Anthony Trollope")
    author_metrics = dict()
    
    for diz in analisi:
        for key in diz:
            try:
                author_metrics[key] += [diz[key]]
            except:
                author_metrics[key] = [diz[key]]
                

    for key in author_metrics:
        if type(author_metrics[key][0]) != list:
            author_metrics[key] = (statistics.mean(author_metrics[key]), statistics.stdev(author_metrics[key]))
        else:
            tot_el = len(author_metrics[key])
            author_metrics[key] = (sc.parallelize(author_metrics[key])
                        .flatMap(lambda x: x)
                        .map(lambda x: (x[0], [x[1]]))
                        .reduceByKey(lambda a,b: a+b).map(lambda x: (x[0], mean_std_couple(x[1], tot_el)))
                       ).collect()
    print(author_metrics)

{'vocabulary_size': (1586, 39.59797974644666), 'text_length': (7877, 316.7838379715733), 'V_T': (0.20140747884819438, 0.0030728265074600365), 'hentropy': (8.519433709418967, 0.1222267318058073), 'prob_distr_of_30': [('i', (0.039899364086489864, 0.004499655634472745)), ('of', (0.02385766442425001, 0.003618728366022868)), ('in', (0.017722612916154123, 0.0025189293873621715)), ('was', (0.01616152813187448, 0.0012351823658520702)), ('he', (0.011169968304732653, 8.939637647902112e-05)), ('but', (0.009155159609214989, 0.0007272608509579331)), ('there', (0.002352018816150529, 0.0033262569087567895)), ('have', (0.0031477595358597706, 0.004451604226702126)), ('would', (0.002468831008517467, 0.0034914542954526478)), ('the', (0.05117500693945394, 0.008803923500961045)), ('and', (0.03369506723983677, 0.0005300482015960147)), ('to', (0.025836043910093455, 0.0030903225576665847)), ('a', (0.02060482714368237, 0.0012360263734891516)), ('that', (0.018837199950778226, 0.003910401437686302)), ('you', (0.

In [26]:
    # genero le metriche dello stesso autore
    
    #test_metrics = generate_metrics("datasets/Anthony Trollope___The Relics of General Chasse.txt") 
    
    test_metrics = generate_metrics("datasets/Andrew Lang___How to Fail in Literature.txt")

Caricamento del file ...  caricamento completato
Calcolo le metriche attendere ...  calcolo completato


In [30]:
    # supponendo una distribuzione normalizzata 
    # calcolo un valore compreso fra 0 e 1 secondo la funzione di distribuzione di probabilità
    
    import numpy as np
    
    def gaussian(x, mu, sigma):
        if sigma != 0:
            return np.exp(-np.power((x - mu)/sigma, 2)/2)
        return 1 if x == mu else 0
    
    def search_tuple(_list, value):
        for tup in _list:
            if tup[0] == value:
                return tup
        return None
        
    score = 0
    total = 0
    
    for key in test_metrics:
        if type(test_metrics[key]) != list:
            score += gaussian(test_metrics[key], author_metrics[key][0], author_metrics[key][1])
            total += 1
        else:
            pass
            for tup in test_metrics[key]:
                res_search = search_tuple(author_metrics[key], tup[0])
                
                if res_search != None:
                    score += gaussian(tup[1], res_search[1][0], res_search[1][1])
                    total += 1
                
                else:
                    # penalita' ?????
                    total += 1
    
    print(score/total * 100, "%")

43.34111533843908 %
